In [25]:
import os
import json
import random
import numpy as np
import pandas as pd

import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from tqdm.notebook import tqdm

from sklearn.metrics import root_mean_squared_error, r2_score
from xgboost import XGBRegressor
import joblib

from src.dataset import PropertyDataset
from src.models import (
     LateFusionModel,
    DualZoomResidualCNN,
    MultiScaleResidualFusion
)

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

OUTPUT_DIR = "outputs"
os.makedirs(OUTPUT_DIR, exist_ok=True)

METRICS_PATH = os.path.join(OUTPUT_DIR, "metrics.json")

In [26]:
def set_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

set_seed(42)

In [27]:
def evaluate(y_true, y_pred):
    return {
        "rmse": float(root_mean_squared_error(y_true, y_pred)),
        "r2": float(r2_score(y_true, y_pred))
    }

def update_metrics(model_name, values, path=METRICS_PATH):
    if os.path.exists(path):
        with open(path, "r") as f:
            data = json.load(f)
    else:
        data = {}

    data[model_name] = values

    with open(path, "w") as f:
        json.dump(data, f, indent=4)

In [28]:
train_df = pd.read_csv("data/processed/train_processed.csv")
val_df   = pd.read_csv("data/processed/val_processed.csv")

TAB_COLS = [
    c for c in train_df.columns
    if c not in ["id", "log_price", "lat", "long"]
]

y_train = train_df["log_price"].values
y_val   = val_df["log_price"].values

In [29]:
xgb = XGBRegressor(
    n_estimators=400,
    max_depth=6,
    learning_rate=0.05,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=42,
    n_jobs=-1
)

xgb.fit(train_df[TAB_COLS], y_train)

val_preds = xgb.predict(val_df[TAB_COLS])
metrics_xgb = evaluate(y_val, val_preds)

update_metrics(
    "xgboost",
    {
        "rmse": metrics_xgb["rmse"],
        "r2": metrics_xgb["r2"]
    }
)

print("XGBoost:", metrics_xgb)

joblib.dump(xgb, os.path.join(OUTPUT_DIR, "xgboost_model.pkl"))

XGBoost: {'rmse': 0.27515337403342255, 'r2': 0.7222425901862053}


['outputs\\xgboost_model.pkl']

In [30]:
BATCH_SIZE = 32
EPOCHS = 8
LR = 1e-3

In [31]:
train_ds = PropertyDataset(
    "data/processed/train_processed.csv",
    "data/images",
    split="train",
    mode="fusion"
)

val_ds = PropertyDataset(
    "data/processed/val_processed.csv",
    "data/images",
    split="val",
    mode="fusion"
)
train_loader = DataLoader(train_ds, BATCH_SIZE, shuffle=True, num_workers=4)
val_loader   = DataLoader(val_ds, BATCH_SIZE, shuffle=False, num_workers=4)


In [33]:
model_naive = LateFusionModel(len(TAB_COLS)).to(DEVICE)
optimizer = torch.optim.Adam(model_naive.parameters(), lr=LR)
criterion = nn.MSELoss()

for epoch in range(1, EPOCHS + 1):
    model_naive.train()
    pbar = tqdm(train_loader, desc=f"Naive Fusion Epoch {epoch}/{EPOCHS}")
    for img16, img18, tab, y in pbar:
        img16 = img16.to(DEVICE)
        img18 = img18.to(DEVICE)
        tab   = tab.to(DEVICE)
        y     = y.to(DEVICE)

        optimizer.zero_grad()
        preds = model_naive(img16, img18, tab)
        loss = criterion(preds, y)
        loss.backward()
        optimizer.step()
        pbar.set_postfix(loss=loss.item())


Naive Fusion Epoch 1/8:   0%|          | 0/403 [00:00<?, ?it/s]

Naive Fusion Epoch 2/8:   0%|          | 0/403 [00:00<?, ?it/s]

Naive Fusion Epoch 3/8:   0%|          | 0/403 [00:00<?, ?it/s]

Naive Fusion Epoch 4/8:   0%|          | 0/403 [00:00<?, ?it/s]

Naive Fusion Epoch 5/8:   0%|          | 0/403 [00:00<?, ?it/s]

Naive Fusion Epoch 6/8:   0%|          | 0/403 [00:00<?, ?it/s]

Naive Fusion Epoch 7/8:   0%|          | 0/403 [00:00<?, ?it/s]

Naive Fusion Epoch 8/8:   0%|          | 0/403 [00:00<?, ?it/s]

In [34]:
model_naive.eval()
preds = []

with torch.no_grad():
    for img16, img18, tab, _ in val_loader:
        img16 = img16.to(DEVICE)
        img18 = img18.to(DEVICE)
        tab   = tab.to(DEVICE)

        out = model_naive(img16, img18, tab)
        preds.append(out.cpu().numpy())

preds = np.concatenate(preds)
metrics_naive = evaluate(y_val, preds)

update_metrics(
    "naive_fusion",
    {
        "epochs": EPOCHS,
        "batch_size": BATCH_SIZE,
        "lr": LR,
        "rmse": metrics_naive["rmse"],
        "r2": metrics_naive["r2"]
    }
)

torch.save(
    model_naive.state_dict(),
    "outputs/naive_fusion.pth"
)

print("Saved: outputs/naive_fusion.pth")


Saved: outputs/naive_fusion.pth


In [35]:
EPOCHS = 15
LR = 1e-4

In [36]:
train_df["xgb_pred"] = xgb.predict(train_df[TAB_COLS])
val_df["xgb_pred"]   = xgb.predict(val_df[TAB_COLS])

train_df.to_csv("data/processed/train_with_xgb.csv", index=False)
val_df.to_csv("data/processed/val_with_xgb.csv", index=False)

In [37]:
train_res_ds = PropertyDataset(
    "data/processed/train_with_xgb.csv",
    "data/images",
    split="train",
    mode="residual",
    xgb_pred_col="xgb_pred"
)

val_res_ds = PropertyDataset(
    "data/processed/val_with_xgb.csv",
    "data/images",
    split="val",
    mode="residual",
    xgb_pred_col="xgb_pred"
)

train_res_loader = DataLoader(train_res_ds, BATCH_SIZE, shuffle=True, num_workers=4)
val_res_loader   = DataLoader(val_res_ds, BATCH_SIZE, shuffle=False, num_workers=4)


In [38]:
model_residual = DualZoomResidualCNN().to(DEVICE)
optimizer = torch.optim.Adam(model_residual.parameters(), lr=LR)

for epoch in range(1, EPOCHS + 1):
    model_residual.train()
    pbar = tqdm(train_res_loader, desc=f"Residual Fusion Epoch {epoch}/{EPOCHS}")
    for img16, img18, res in pbar:
        img16 = img16.to(DEVICE)
        img18 = img18.to(DEVICE)
        res   = res.to(DEVICE)

        optimizer.zero_grad()
        preds = model_residual(img16, img18)
        loss = criterion(preds, res)
        loss.backward()
        optimizer.step()
        pbar.set_postfix(loss=loss.item())


Residual Fusion Epoch 1/15:   0%|          | 0/403 [00:00<?, ?it/s]

Residual Fusion Epoch 2/15:   0%|          | 0/403 [00:00<?, ?it/s]

Residual Fusion Epoch 3/15:   0%|          | 0/403 [00:00<?, ?it/s]

Residual Fusion Epoch 4/15:   0%|          | 0/403 [00:00<?, ?it/s]

Residual Fusion Epoch 5/15:   0%|          | 0/403 [00:00<?, ?it/s]

...

Residual Fusion Epoch 11/15:   0%|          | 0/403 [00:00<?, ?it/s]

Residual Fusion Epoch 12/15:   0%|          | 0/403 [00:00<?, ?it/s]

Residual Fusion Epoch 13/15:   0%|          | 0/403 [00:00<?, ?it/s]

Residual Fusion Epoch 14/15:   0%|          | 0/403 [00:00<?, ?it/s]

Residual Fusion Epoch 15/15:   0%|          | 0/403 [00:00<?, ?it/s]

In [39]:
model_residual.eval()
cnn_residuals = []

with torch.no_grad():
    for img16, img18, _ in val_res_loader:
        img16 = img16.to(DEVICE)
        img18 = img18.to(DEVICE)

        out = model_residual(img16, img18)
        cnn_residuals.append(out.cpu().numpy())

cnn_residuals = np.concatenate(cnn_residuals)

final_preds = val_df["xgb_pred"].values + cnn_residuals

metrics_residual = evaluate(y_val, final_preds)

update_metrics(
    "residual_fusion",
    {
        "epochs": EPOCHS,
        "batch_size": BATCH_SIZE,
        "lr": LR,
        "rmse": metrics_residual["rmse"],
        "r2": metrics_residual["r2"]
    }
)

print("Residual Fusion:", metrics_residual)

torch.save(
    model_residual.state_dict(),
    os.path.join(OUTPUT_DIR, "adaptive_fusion_final.pth")
)


Residual Fusion: {'rmse': 0.25660982046274733, 'r2': 0.7584191450610015}


In [42]:
from src.models import MultiScaleResidualFusion

learn_fusion = MultiScaleResidualFusion(alpha=0.3).to(DEVICE)

optimizer_final = torch.optim.Adam(
    learn_fusion.parameters(), lr=1e-3
)

criterion_final = nn.MSELoss()


In [43]:
model_residual.eval()
learn_fusion.train()

for epoch in range(1, 6):  # 5 epochs is sufficient
    epoch_losses = []
    idx_ptr = 0

    pbar = tqdm(train_res_loader, desc=f"Final Fusion Epoch {epoch}/5")

    for img16, img18, _ in pbar:
        bs = img16.size(0)

        img16 = img16.to(DEVICE)
        img18 = img18.to(DEVICE)

        with torch.no_grad():
            r_hat = model_residual(img16, img18)

        xgb_batch = train_df.iloc[idx_ptr:idx_ptr + bs]["xgb_pred"].values
        y_batch   = train_df.iloc[idx_ptr:idx_ptr + bs]["log_price"].values

        xgb_batch = torch.tensor(xgb_batch, dtype=torch.float32, device=DEVICE)
        y_batch   = torch.tensor(y_batch, dtype=torch.float32, device=DEVICE)

        preds = learn_fusion(xgb_batch, r_hat, r_hat)

        loss = criterion_final(preds, y_batch)

        optimizer_final.zero_grad()
        loss.backward()
        optimizer_final.step()

        epoch_losses.append(loss.item())
        pbar.set_postfix(
            loss=loss.item(),
            alpha=learn_fusion.alpha.item()
        )

        idx_ptr += bs

    print(
        f"Epoch {epoch} | "
        f"Mean Loss: {sum(epoch_losses)/len(epoch_losses):.4f} | "
        f"Alpha: {learn_fusion.alpha.item():.4f}"
    )


Final Fusion Epoch 1/5:   0%|          | 0/403 [00:00<?, ?it/s]

Epoch 1 | Mean Loss: 0.0544 | Alpha: -0.0426


Final Fusion Epoch 2/5:   0%|          | 0/403 [00:00<?, ?it/s]

Epoch 2 | Mean Loss: 0.0483 | Alpha: -0.3306


Final Fusion Epoch 3/5:   0%|          | 0/403 [00:00<?, ?it/s]

Epoch 3 | Mean Loss: 0.0447 | Alpha: -0.5538


Final Fusion Epoch 4/5:   0%|          | 0/403 [00:00<?, ?it/s]

Epoch 4 | Mean Loss: 0.0432 | Alpha: -0.7292


Final Fusion Epoch 5/5:   0%|          | 0/403 [00:00<?, ?it/s]

Epoch 5 | Mean Loss: 0.0425 | Alpha: -0.8506


In [44]:
learn_fusion.eval()
final_preds = []

idx_ptr = 0
pbar = tqdm(val_res_loader, desc="Final Fusion Validation")

with torch.no_grad():
    for img16, img18, _ in pbar:
        bs = img16.size(0)

        img16 = img16.to(DEVICE)
        img18 = img18.to(DEVICE)

        r_hat = model_residual(img16, img18)

        xgb_batch = val_df.iloc[idx_ptr:idx_ptr + bs]["xgb_pred"].values
        xgb_batch = torch.tensor(
            xgb_batch, dtype=torch.float32, device=DEVICE
        )

        preds = learn_fusion(xgb_batch, r_hat, r_hat)
        final_preds.append(preds.cpu().numpy())

        idx_ptr += bs

final_preds = np.concatenate(final_preds)

metrics_final = evaluate(y_val, final_preds)

update_metrics(
    "learned_fusion",
    {
        "alpha": learn_fusion.alpha.item(),
        "rmse": metrics_final["rmse"],
        "r2": metrics_final["r2"]
    }
)

print("LEARN MODEL METRICS:", metrics_final)
print("Learned alpha:", learn_fusion.alpha.item())


Final Fusion Validation:   0%|          | 0/101 [00:00<?, ?it/s]

LEARN MODEL METRICS: {'rmse': 0.2701565372495089, 'r2': 0.7322392410259168}
Learned alpha: -0.8506068587303162


In [45]:
torch.save(
    {
        "residual_cnn": model_residual.state_dict(),
        "fusion_head": learn_fusion.state_dict(),
        "xgb_model": os.path.join(OUTPUT_DIR, "xgboost_model.pkl"),
    },
    os.path.join(OUTPUT_DIR, "learned_model.pth")
)

print("Saved learned_model.pth")

Saved learned_model.pth
